In [39]:
import pandas as pd
import numpy as np


In [40]:
# Abstand zwischen den gegenüberliegenden Side Planes
SideOffset = 1.48

# Offset wie weit die Top Plane von Z=0 entfernt ist (Z=0 ist die unterste Kante des Side Planes)
TopOffset = 1.74

In [53]:
def calculate_distance(df, name1, name2, x_col, y_col, z_col, print_values=True):
    point1 = df[df["Mic_Index"] == name1][[x_col, y_col, z_col]].iloc[0]
    point2 = df[df["Mic_Index"] == name2][[x_col, y_col, z_col]].iloc[0]
    
    # euklidische Distanz berechnen
    dist = np.sqrt((point2[x_col] - point1[x_col])**2 + 
                   (point2[y_col] - point1[y_col])**2 + 
                   (point2[z_col] - point1[z_col])**2)
    
    if print_values == True:
        print(name1)
        print(x_col, ":", round(point1[x_col], 3))
        print(y_col, ":", round(point1[y_col], 3))
        print(z_col, ":", round(point1[z_col], 3))
        print("")
        print(name2)
        print(x_col, ":", round(point2[x_col], 3))
        print(y_col, ":", round(point2[y_col], 3))
        print(z_col, ":", round(point2[z_col], 3))

        print("distance = ", round(dist, 3))


    return dist

In [52]:

file_path = r"mic_positions\side_postions_measured.xlsx"
df = pd.read_excel(file_path, engine='openpyxl')

# cut empty rows
df = df.head(64)

    Unnamed: 0 Mic_Index  Plane      X      Y  X measured  Y measured  \
0          0.0        B1  right  0.649  0.024       0.649       0.020   
1          1.0        B2  right  0.892  0.266       0.902       0.266   
2          2.0        B3  right  0.252  0.287       0.251       0.287   
3          3.0        B4  right  1.312  0.385       1.313       0.380   
4          4.0        B5  right  0.561  0.429       0.562       0.425   
..         ...       ...    ...    ...    ...         ...         ...   
59        59.0        A9    top    NaN    NaN       0.065       0.778   
60        60.0       A10    top    NaN    NaN       0.790       0.906   
61        61.0       A11    top    NaN    NaN       0.485       1.144   
62        62.0       A12    top    NaN    NaN       1.138       1.170   
63        63.0       A13    top    NaN    NaN       0.198       1.182   

   Unnamed: 7  delta x  delta y  
0         NaN    0.000    0.004  
1         NaN   -0.010    0.000  
2         NaN    0.00

In [ ]:


################ TEMP BIS TOP IN DEN DATEN IST

#temp_top_data = [

#    {"Unnamed: 0": "52" , "Mic_Index": "A2" , "Plane": "top", "X": -0.145  , "Y": -0.132  },
#    {"Unnamed: 0": "51" , "Mic_Index": "A1" , "Plane": "top", "X": 0.0     , "Y": 0.0     },
#    {"Unnamed: 0": "53" , "Mic_Index": "A3" , "Plane": "top", "X": 0.024   , "Y": 0.276   },
#    {"Unnamed: 0": "54" , "Mic_Index": "A4" , "Plane": "top", "X": 0.207   , "Y": -0.27   },
#    {"Unnamed: 0": "55" , "Mic_Index": "A5" , "Plane": "top", "X": -0.386  , "Y": 0.068   },
#    {"Unnamed: 0": "56" , "Mic_Index": "A6" , "Plane": "top", "X": 0.37    , "Y": 0.235   },
#    {"Unnamed: 0": "57" , "Mic_Index": "A7" , "Plane": "top", "X": -0.125  , "Y": -0.464  },
#    {"Unnamed: 0": "58" , "Mic_Index": "A8" , "Plane": "top", "X": -0.239  , "Y": 0.46    },
#    {"Unnamed: 0": "59" , "Mic_Index": "A9" , "Plane": "top", "X": 0.521   , "Y": -0.19   },
#    {"Unnamed: 0": "60" , "Mic_Index": "A10" , "Plane": "top", "X": -0.544  , "Y": -0.224  },
#    {"Unnamed: 0": "61" , "Mic_Index": "A11", "Plane": "top", "X": 0.263   , "Y": 0.562   },
#    {"Unnamed: 0": "62" , "Mic_Index": "A12", "Plane": "top", "X": 0.195   , "Y": -0.621  },
#    {"Unnamed: 0": "63" , "Mic_Index": "A13", "Plane": "top", "X": 0.691   , "Y": 0.152   },
#    {"Unnamed: 0": "64" , "Mic_Index": "A14", "Plane": "top", "X": -0.422  , "Y": -0.6    },
#    {"Unnamed: 0": "65" , "Mic_Index": "A15", "Plane": "top", "X": 0.6     , "Y": -0.506  },
#    {"Unnamed: 0": "66" , "Mic_Index": "A16", "Plane": "top", "X": 0.59    , "Y": 0.587   },
#    {"Unnamed: 0": "67" , "Mic_Index": "A17", "Plane": "top", "X": -0.562  , "Y": 0.673   },
#]
#temp_top_df = pd.DataFrame(temp_top_data)
#temp_top_df.loc[temp_top_df["Plane"] == "top", "X"] = temp_top_df["X"] + 0.7
#temp_top_df.loc[temp_top_df["Plane"] == "top", "Y"] = temp_top_df["Y"] + 0.7

#df = pd.concat([df, temp_top_df], ignore_index=True)

################ TEMP OVER

# print(df)

new_df = df.copy()

new_df.loc[new_df["Plane"] == "left", "X"] = 0
new_df.loc[new_df["Plane"] == "left", "Y"] = df["X"]        + SideOffset - 1.4
new_df.loc[new_df["Plane"] == "left", "Z"] = df["Y"]

new_df.loc[new_df["Plane"] == "front", "X"] = -df["X"]+1.4  + SideOffset - 1.4
new_df.loc[new_df["Plane"] == "front", "Y"] = 0
new_df.loc[new_df["Plane"] == "front", "Z"] = df["Y"]

new_df.loc[new_df["Plane"] == "right", "X"] = SideOffset 
new_df.loc[new_df["Plane"] == "right", "Y"] = -df["X"]+1.4  + SideOffset - 1.4
new_df.loc[new_df["Plane"] == "right", "Z"] = df["Y"]

new_df.loc[new_df["Plane"] == "top", "X"] = df["X"]         + (SideOffset - 1.4) / 2
new_df.loc[new_df["Plane"] == "top", "Y"] = df["Y"]         + (SideOffset - 1.4) / 2
new_df.loc[new_df["Plane"] == "top", "Z"] = TopOffset


columns_order = [
    "Unnamed: 0",
    "Mic_Index",
    "Plane",
    "X",
    "Y",
    "Z",
    "X measured",
    "Y measured",
]
new_df = new_df[columns_order]


# Ergebnis anzeigen
print(new_df)


In [54]:
_ = calculate_distance(new_df, "D9", "B6", x_col="X", y_col="Y", z_col="Z")

D9
X : 0.0
Y : 0.62
Z : 0.773
B6
X : 1.48
Y : 0.576
Z : 0.63
distance =  1.488


In [44]:
# fancy interactiver plot 
# damit kann man die namen der mikrofone sehr gut sehen


import plotly.express as px

fig = px.scatter_3d(new_df, x='X', y='Y', z='Z', text='Mic_Index', title='3D Scatter Plot')

# Plot anzeigen
fig.show()